<a href="https://colab.research.google.com/github/hongsunjang/YBIGTA-pre-session/blob/main/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

In [147]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [148]:
train = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/titanic_train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/titanic_test.csv')
submission = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/titanic_submission.csv')

In [81]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [82]:
test.head()

,Unnamed: 0,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [83]:
train.isnull().sum() / train.shape[0]

PassengerId    0.000000
Survived       0.000000
Pclass         0.000000
Name           0.000000
Sex            0.000000
Age            0.198653
SibSp          0.000000
Parch          0.000000
Ticket         0.000000
Fare           0.000000
Cabin          0.771044
Embarked       0.002245
dtype: float64

In [79]:

train_drop = train.drop(columns=["Age", "Cabin", "Embarked"])

In [80]:
train_drop.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
SibSp          0
Parch          0
Ticket         0
Fare           0
dtype: int64

In [84]:
train_drop.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
dtype: object

In [87]:
train_drop[["Name","Sex", "Ticket"]].nunique()

Name      891
Sex         2
Ticket    681
dtype: int64

In [88]:
train_drop = pd.get_dummies(train_drop, columns=['Sex'])

In [89]:
# "Name", "Tiket", "Cabin"은 많으니가 drop
train_drop = train_drop.drop(columns = ["Name", "Ticket"])

In [90]:
train_drop.head()

,PassengerId,Survived,Pclass,SibSp,Parch,Fare,Sex_female,Sex_male
0,1,0,3,1,0,7.2500,0,1
1,2,1,1,1,0,71.2833,1,0
2,3,1,3,0,0,7.9250,1,0
3,4,1,1,1,0,53.1000,1,0
4,5,0,3,0,0,8.0500,0,1


In [91]:
X_train = train_drop.drop(columns = ["Survived"])
y_train = train_drop["Survived"]
y_train.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [92]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state=42, stratify=y_train)

In [94]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()

dtc.fit(X_train, y_train)

pred_val = dtc.predict(X_val)

In [95]:
from sklearn.metrics import f1_score

score = f1_score(y_val, pred_val)
score

0.6575342465753424

In [96]:
from sklearn.model_selection import GridSearchCV

dtc = DecisionTreeClassifier()

max_depth = [1, 5, 10, 20, 40]

max_leaf_nodes = [1, 3, 5, 10, 20, 40]

param_grid = {"max_depth": max_depth, "max_leaf_nodes": max_leaf_nodes}

In [97]:
gs =GridSearchCV(dtc, param_grid, scoring = "f1", cv = 3)

gs.fit(X_train, y_train)

gs.best_params_

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
15 fits failed out of a total of 90.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py", line 942, in fit
    X_idx_sorted=X_idx_sorted,
  File "/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py", line 316, in fit
    max_leaf_nodes
ValueError: max_leaf_nodes 1 must be either None or larger than 1

  warnings.warn(som

{'max_depth': 1, 'max_leaf_nodes': 3}

In [101]:
dtc = DecisionTreeClassifier(max_depth= gs.best_params_["max_depth"], max_leaf_nodes= gs.best_params_["max_leaf_nodes"])

dtc.fit(X_train, y_train)

pred_val = dtc.predict(X_val)

score = f1_score(pred_val, y_val)

score

0.6923076923076924

# Test 수행

In [102]:
train_drop.head()

,PassengerId,Survived,Pclass,SibSp,Parch,Fare,Sex_female,Sex_male
0,1,0,3,1,0,7.2500,0,1
1,2,1,1,1,0,71.2833,1,0
2,3,1,3,0,0,7.9250,1,0
3,4,1,1,1,0,53.1000,1,0
4,5,0,3,0,0,8.0500,0,1


In [103]:
test.head()

,Unnamed: 0,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [106]:
test_drop = test.drop(columns=["Name", "Age", "Ticket", "Cabin", "Embarked"],)
test_drop = pd.get_dummies(test_drop, columns=['Sex'])
test_drop.head()
#PassengerId	Survived	Pclass	Age	SibSp	Parch	Fare	Sex_female	Sex_male	Embarked_C	Embarked_Q	Embarked_S

,Unnamed: 0,PassengerId,Pclass,SibSp,Parch,Fare,Sex_female,Sex_male
0,0,892,3,0,0,7.8292,0,1
1,1,893,3,1,0,7.0000,1,0
2,2,894,2,0,0,9.6875,0,1
3,3,895,3,0,0,8.6625,0,1
4,4,896,3,1,1,12.2875,1,0


In [126]:
test_drop.isnull().sum()
#test_drop[test_drop["Fare"].isnull() == True]

Unnamed: 0     0
PassengerId    0
Pclass         0
SibSp          0
Parch          0
Fare           0
Sex_female     0
Sex_male       0
dtype: int64

In [125]:
test_drop.fillna(test_drop.median(), inplace =True)
#test_drop[test_drop["Fare"].isnull() == True ]["Fare"] = test_drop.median()["Fare"]

In [127]:
test_drop.dtypes

Unnamed: 0       int64
PassengerId      int64
Pclass           int64
SibSp            int64
Parch            int64
Fare           float64
Sex_female       uint8
Sex_male         uint8
dtype: object

In [133]:
test_drop.drop(columns = ["Unnamed: 0"],inplace= True)
test_drop.isnull().sum()

PassengerId    0
Pclass         0
SibSp          0
Parch          0
Fare           0
Sex_female     0
Sex_male       0
dtype: int64

In [134]:
X_test = test_drop[:]

In [149]:
pred_test =dtc.predict(X_test)
submission["Survived"] = pred_test
submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [151]:
submission.to_csv('/content/drive/MyDrive/Colab_Notebooks/final_submission.csv', index=False)